**Chat Assistant for SQLite Database**

In [38]:
pip install streamlit as st

  Using cached streamlit-1.42.0-py2.py3-none-any.whl.metadata (8.9 kB)
ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [39]:
import sqlite3
import re

#Connect to sqlite database
conn = sqlite3.connect("your_database_name.db")

#Create a cursor object
cursor = conn.cursor()

In [40]:
def create_database(db_name):
    import sqlite3
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("""CREATE TABLE IF NOT EXISTS Employees (
                        ID INTEGER PRIMARY KEY,
                        Name TEXT NOT NULL,
                        Department TEXT NOT NULL,
                        Salary INTEGER NOT NULL,
                        Hire_Date TEXT NOT NULL
                    )""")
    conn.commit()
    conn.close()


In [41]:
#Create Employees table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Employees (
      ID INTEGER PRIMARY KEY,
      Name TEXT NOT NULL,
      Department TXT NOT NULL,
      Salary INTEGER NOT NULL,
      Hire_Date TEXT NOT NULL
      )
""")

In [42]:
#Create Departments table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Departments (
      ID INTEGER PRIMARY KEY,
      Name TEXT NOT NULL UNIQUE,
      Manager TEXT NOT NULL
    )
""")


In [43]:
#Insert sample data

cursor.executemany("""
    INSERT OR IGNORE INTO Employees (ID, Name, Department, Salary, Hire_Date)
    VALUES (?, ?, ?, ?, ?)
""",[
    (1, 'Alice', 'Sales', 50000, '2021-01-15'),
    (2, 'Bob', 'Engineering', 70000, '2020-06-10'),
    (3, 'Charlie', 'Marketing', 60000, '2022-03-20')
])

cursor.executemany("""
    INSERT OR IGNORE INTO Departments (ID, Name, Manager)
    VALUES (?, ?, ?)
""",[
    (1, 'Sales', 'Alice'),
    (2, 'Engineering', 'Bob'),
    (3, 'Marketing', 'Charlie')
])

In [44]:
conn.commit()
conn.close()

In [45]:
# Function to query the database

def query_database(sql, params=()):
    try:
        conn = sqlite3.connect("your_database_name.db")
        cursor = conn.cursor()
        cursor.execute(sql, params)
        results = cursor.fetchall()
        conn.close()
        return results
    except sqlite3.DatabaseError as e:
        print(f"Database error: {e}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Function to process user input

def process_query(user_input):
    user_input = user_input.lower()

# Match user query patterns

    if match := re.search(r"show me all employees in the (.+) department", user_input):
        department = match.group(1).capitalize()
        return query_database("SELECT Name FROM Employees WHERE Department = ?", (department,))

    elif match := re.search(r"who is the manager of the (.+) department", user_input):
        department = match.group(1).capitalize()
        return query_database("SELECT Manager FROM Departments WHERE Name = ?", (department,))

    elif match := re.search(r"list all employees hired after (\d{4}-\d{2}-\d{2})", user_input):
        date = match.group(1)
        return query_database("SELECT Name FROM Employees WHERE Hire_Date > ?", (date,))

    elif match := re.search(r"what is the total salary expense for the (.+) department", user_input):
        department = match.group(1).capitalize()
        result = query_database("SELECT SUM(Salary) FROM Employees WHERE Department = ?", (department,))
        return result[0][0] if result else "No salary data found."

    else:
        return "I'm sorry, I didn't understand that query."

# Example usage

user_input = "Show me all employees in the engineering department"
result = process_query(user_input)
print(result)


[('Bob',)]


In [46]:
# Chat function to interact with the user

def chat():
    print("\n Welcome to the Employee Chat Assistant! Type 'exit' to quit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print(" Goodbye!")
            break
        response = process_query(user_input)
        print("Assistant:", response if response else "No results found.")

# Run the script
if __name__ == "__main__":
    create_database("your_database_name.db")  # Ensure database is created before running
    chat()


 Welcome to the Employee Chat Assistant! Type 'exit' to quit.
You: exit
 Goodbye!
